In [1]:
from itertools import combinations as subsets
import random,time

# An Introduction to Computational Algebraic Geometry using Sage

**Presented by:** Greg DePaul on behalf of the UC Davis SIAM. 

**Materials:** David A. Cox • John Little • Donal O’Shea - "Ideals, Varieties, and Algorithms" -
Chapters 1,2 and 3.1

**Attribution:** Alex Chandler's Math 165 Course. Much of the materials, such as examples, come directly from this course. 

In [2]:
# Useful Functions:

def is_less_than(a,b,order):
    if order == 'lex':
        return next((k for (i,j) in zip(a,b) if (k := i - j) != 0), 0) < 0
    elif order == 'deglex':
        return (s := sum(a) - sum(b)) < 0 or (s == 0 and next((k for (i,j) in zip(a,b) if (k := i - j) != 0), 0) < 0)
    elif order == 'degrevlex':
        return (s := sum(a) - sum(b)) < 0 or (s == 0 and next((k for (i,j) in zip(a[::-1],b[::-1]) if (k := i - j) != 0), 0) > 0)
    else:
        raise Exception('invalid ordering mode')
        
def lt(f,order):
    ret = f.monomials()[0]
    for m in f.monomials():
        if is_less_than(ret.degrees(), m.degrees(), order):
            ret = m
    return f.coefficient(ret) * ret

def div_alg_multi(R,f,poly_list,order):
    lto = lambda p: lt(p, order)
    q_list = [0] * len(poly_list)
    r = 0
    while f != 0:
        for i, g in enumerate(poly_list):
            if lto(g).divides(lto(f)):
                q_list[i] += lto(f)//lto(g)
                f -= (lto(f)//lto(g))*g
                break
        else:
            r += lto(f)
            f -= lto(f)
    return (q_list, r)

def s_polynomial(R,f,g,order):
    return (xg := lcm(f0:=lt(f,order), g0:=lt(g, order)))//f0*f - xg//g0*g

def is_groebner_basis(R,poly_list,order):
    def s_poly(f,g):
        return (xg := lcm(f0:=lt(f,order), g0:=lt(g, order)))//f0*f - xg//g0*g
    for f in poly_list:
        for g in poly_list:
            if f != g and div_alg_multi(R,s_poly(f, g),poly_list,order)[1] != 0:
                return False
    return True

def S(R, f,g,order):
    mdeg_f = lt(f,order).degrees()
    mdeg_g = lt(g,order).degrees()
    lcm = product([R.gens()[i] ^ max(mdeg_f[i], mdeg_g[i]) for i in range(len(mdeg_f))])
    return R((lcm/lt(f,order))*f - ((lcm)/lt(g,order))*g )
    
def is_groebner_basis(R,poly_list,order):
    GB = True
    for f,g in subsets(poly_list,2):
        S_poly = S(R, f,g,order)
        if (div_alg_multi(R, S_poly, poly_list, order)[1]) != 0:
            return False
    return True


def groebner_basis(R,poly_list,order):
    my_ideal = ideal(poly_list)
    return my_ideal.groebner_basis('toy:buchberger')

def minimal_groebner_basis(R,poly_list,order):
    my_ideal = ideal(poly_list)
    return my_ideal.groebner_basis('toy:buchberger2')

def is_reduced_groebner_basis(R,G,order):
    my_ideal = ideal(poly_list)
    return my_ideal.reduced()

def in_ideal_gen_by(R, f, poly_list, order):
    G = minimal_groebner_basis(R, poly_list, order)
    return div_alg_multi(R,f,G,order)[1] == 0

# Multivariate Polynomial Rings

In [3]:
R.<x,y> = QQ[]
f = 1 + x*y + x^2
g = y + x*y^2
h = y + x*y + x*y^2
I = ideal(f,g,h)
I

Ideal (x^2 + x*y + 1, x*y^2 + y, x*y^2 + x*y + y) of Multivariate Polynomial Ring in x, y over Rational Field

# Orderings of Monomials

**Definition** A *monomial ordering* $>$ on $k[x_1, \ldots , x_n]$ is a relation $>$ on $\mathbb Z_n \geq 0$, or equivalently, a relation on the set of monomials $x^{\alpha}, \alpha \in \mathbb Z_{\geq 0}^n$, satisfying:
1. $>$ is a total (or linear) ordering on $\mathbb Z_{\geq 0}^n$.
2. If $\alpha > \beta$ and $\gamma \in \mathbb Z_{\geq 0}^n$, then $\alpha + \gamma > \beta + \gamma$.
3. $>$ is a well-ordering on $\mathbb Z_{\geq 0}^n$. This means that every nonempty subset of $\mathbb Z_{\geq 0}^n$ has a smallest element under $>$. In other words, if $A \subset \mathbb Z_{\geq 0}^n$ is nonempty, then there is $\alpha \in A$ such that $\beta > \alpha$ for every $\beta \not = \alpha$ in $A$.

**Definition (Lexicographic Order).** Let $\alpha = (\alpha_1, \ldots, \alpha_n)$ and $\beta = (\beta_1, \ldots, \beta_n)$ be in $\mathbb Z_{\geq 0}^n$. We say $\alpha >_{lex} \beta$ if the leftmost nonzero entry of the vector difference $ \alpha - \beta \in \mathbb Z^n$ is positive.We will write $x^{\alpha} >_{lex} x^{\beta}$ if $\alpha >_{lex} \beta$.

In [4]:
# Example 1
is_less_than([0,3,4],[1,2,0],'lex')

True

In [6]:
# Example 2
is_less_than([0,0,3],[0,0,4],'lex')

True

**Definition 5 (Graded Lex Order).** Let α, β ∈ Zn≥0 . We say $\alpha >_{deglex} \beta$ if
$$|\alpha| = \sum_{i = 1}^n \alpha_i > |\beta| = \sum_{i = 1}^n |\beta_i| \ \ \ \text{ or } \ \ \ |\alpha| = |\beta| \text{ and } \alpha >_{lex} \beta$$

In [8]:
# Example 1
is_less_than([3,2,0],[1,2,3],'deglex')

True

In [7]:
# Example 1.5
is_less_than([3,0,0],[0,0,1000],'deglex')

True

In [9]:
# Example 2
is_less_than([1,1,5],[1,2,4],'deglex')

True

**Definition 6 (Graded Reverse Lex Order).** Let $\alpha, \beta \in \mathbb Z^n_{\geq 0}$. We say $\alpha >_{degrevlex} \beta$ if
$$|\alpha| = \sum_{i = 1}^n \alpha_i > |\beta| = \sum_{i = 1}^n |\beta_i| \ \ \ \text{ or } \ \ \ |\alpha| = |\beta| \text{ and } \text{and the rightmost nonzero entry of } \alpha - \beta \in \mathbb Z^n \text{ is negative}$$

In [10]:
# Example 1
is_less_than([4,2,3],[4,7,1],'degrevlex')

True

In [11]:
# Example 2
is_less_than([4,1,3],[1,5,2],'degrevlex')

True

# Describing a Single Polynomial

**Definition** The *multidegree* of $f$ is $\text{multideg}(f) = \max( \alpha \in \mathbb Z^n_{\geq 0} |  a_{\alpha} \not = 0 )$

**Definition** The *leading coefficient* of $f$ is $\text{LC}(f) = a_{\text{multideg}(f)} \in k$

**Definition** The *leading monomial* of $f$ is $\text{LM}(f) = x^{\text{multideg}(f)}$

**Definition** The *leading term* of $f$ is $LT(f) = LC(f) \cdot LM(f)$

In [12]:
# Example 1
R.<x,y,z> = PolynomialRing(QQ)
f = x^2 * y * z^2 + x * y^3 * z + x^3
print(f"the leading term of f in lex order is {lt(f,'lex')}")
print(f"the leading term of f in deglex order is {lt(f,'deglex')}")
print(f"the leading term of f in degrevlex order is {lt(f,'degrevlex')}")

the leading term of f in lex order is x^3
the leading term of f in deglex order is x^2*y*z^2
the leading term of f in degrevlex order is x*y^3*z


In [13]:
# Example 2
g = 4*x*y^2*z + 4*z^2 - 5*x^3 + 7*x^2*z^2
print(f"the leading term of g in lex order is {lt(g,'lex')}")
print(f"the leading term of g in deglex order is {lt(g,'deglex')}")
print(f"the leading term of g in degrevlex order is {lt(g,'degrevlex')}")

the leading term of g in lex order is -5*x^3
the leading term of g in deglex order is 7*x^2*z^2
the leading term of g in degrevlex order is 4*x*y^2*z


# Multivariate Division Algorithm

**Theorem (Division Algorithm in $k[x_1, \ldots , x_n]$).** Let $>$ be a monomial order on $\mathbb Z^n_{\geq 0}$, and let $F = \langle f_1, \ldots,f_s\rangle$ be an ordered $s$-tuple of polynomials in $k[x_1, \ldots , x_n]$. Then every $f \in k[x_1, \ldots ,x_n]$ can be written as
$$f = q_1 f_1 + \ldots + q_s f_s + r,$$
where $q_i, r \in k[x_1, \ldots , x_n]$, and either $r = 0$ or $r$ is a linear combination, with coefficients in $k$, of monomials, none of which is divisible by any of $LT(f_1), \ldots, LT( f_s)$. We call $r$ a *remainder* of $f$ on division by $F$. Furthermore, if $q_if_i  \not = 0$, then
$$\text{multideg}( f ) \geq \text{multideg}(qi,fi ).$$

In [15]:
# Example 1
R.<x,y> = PolynomialRing(QQ)
f = x*y^2 + 1
g = x*y + 1
h = y + 1
poly_list = [g,h]
div_alg_multi(R,f,poly_list,'lex')

([y, -1], 2)

In [16]:
# Example 2
R.<x,y> = PolynomialRing(QQ)
f = x^2*y+x*y^2+y^2
g = x*y-1
h = y^2-1
poly_list = [g,h]
div_alg_multi(R,f,poly_list,'lex')

([x + y, 1], x + y + 1)

In [17]:
# Example 3
R.<x,y> = PolynomialRing(QQ)
f = x^2*y+x*y^2+y^2
g = y^2-1
h = x*y-1
poly_list = [g,h]
div_alg_multi(R,f,poly_list,'lex')

([x + 1, x], 2*x + 1)

In [18]:
# Example 4
R.<x,y> = PolynomialRing(QQ)
f = R(2*x*(x*y+y)**9+8*y*(x**2-y**2)**7+2*x+3*y**2)
g = R((x*y+y)**9)
h = R((x**2-y**2)**7) 
poly_list = [g,h]
div_alg_multi(R,f,poly_list,'lex')

([2*x, 8*y], 3*y^2 + 2*x)

# S-Polynomial

**Definition** Let $f , g \in k[x_1, \ldots , x_n]$ be nonzero polynomials.

(1) If $\text{multideg}(f) = \alpha$ and $\text{multideg}(g) = \beta$, then let $\gamma = (\gamma_1, \ldots,\gamma_n)$, where
$$\gamma_i = max(\alpha_i, \beta_i)$$ 
for each $i$. We call $x^{\gamma}$ the least common multiple of $LM(f)$ and $LM(g)$, written $x^{\gamma} = lcm(LM( f ), LM(g))$.

(2)  The *S-polynomial* of f and g is the combination
$$S(f,g) = \frac{x^{\gamma}}{LT(f)} \cdot f - \frac{x^{\gamma}}{LT(g)} \cdot g$$

In [19]:
# Example 1
Q.<x,y> = PolynomialRing(QQ)
f = x^3*y^2 - x^2*y^3 + x
g = 3*x^4*y + y^2
print(f"the S-polynomial of f and g is {S(Q,f,g,'deglex')}")

the S-polynomial of f and g is -x^3*y^3 - 1/3*y^3 + x^2


# Buchberger's Criterion

**Definition.** Fix a monomial order on the polynomial ring $k[x_1, \ldots , x_n]$. A finite subset $G = {g_1,\ldots,g_t}$ of an ideal $I \subset k[x_1, \ldots,x_n]$ different from $\{0\}$ is said to be a *Gröbner basis* (or standard basis) if
$$\langle LT(g_1), \ldots , LT(g_t) \rangle = \langle LT(I) \rangle.$$
Using the convention that $\langle \emptyset\rangle = \{0\}$, we define the empty set $\emptyset$ to be the Gröbner
basis of the zero ideal $\{0\}$.

**Theorem** Fix a monomial order. Then every ideal $I \subset k[x_1 , \ldots, x_n ]$ has a Gröbner basis. Furthermore, any Gröbner basis for an ideal $I$ is a basis of $I$.

**Theorem (Buchberger’s Criterion).** Let $I$ be a polynomial ideal. Then a basis $G = \{g_1, \ldots,g_t \}$ of $I$ is a Gröbner basis of $I$ if and only if for all pairs $i \not  = j$, the remainder on division of $S(g_i, g_j)$ by $G$ (listed in some order) is zero.

In [20]:
# show [f1,f2] is not a Grobner basis
Q.<x,y,z> = PolynomialRing(QQ)
f1 = y - x^2
f2 = z - x^3
is_groebner_basis(Q,[f1,f2],'lex')

False

In [24]:
# show [f1,f2] is a Grobner basis
f1 = x - z^2
f2 = y - z^3
is_groebner_basis(Q,[f1,f2],'lex')

True

In [25]:
# show [g1,g2,g3] is a Grobner basis
g1 = x^4*y^2 - z^5
g2 = x^3*y^3 - 1
g3 = x^2*y^4 - 2*z
is_groebner_basis(Q,[g1, g2, g3],'deglex')

False

In [23]:
# show [f1,f2,f3,f4,f5] is a Grobner basis
f1 = x^3 - 2*x*y
f2 = x^2*y - 2*y^2 + x
f3 = -x^2 
f4 = -2*x*y
f5 = -2*y^2 + x
is_groebner_basis(Q,[f1, f2, f3, f4, f5],'deglex')

True

In [26]:
# show [f1,f2] is a Grobner basis
Q.<x,y> = PolynomialRing(QQ)
f1 = x^2 + 2*x + 3 + y^5 - y
f2 = y^6 - y^2 + 2*y
is_groebner_basis(Q,[f1,f2],'lex')

True

## Buchberger's Algorithm

**Theorem 2 (Buchberger’s Algorithm).** Let $I = \langle f1 , . . . , fs \rangle  \not = \{0\}$ be a polynomial ideal. Then a Gröbner basis for $I$ can be constructed in a finite number of steps.

In [27]:
R.<x,y> = PolynomialRing(QQ)
f = R(x^3 - 2*x*y)
g = R(x^2*y - 2*y^2 + x)
G = groebner_basis(R, [f,g], 'deglex')

print(f"Proposed Gröbner basis for <f1,f1>: {G}")
print(f"This is a Gröbner basis: {is_groebner_basis(R,G,'deglex')}")

Proposed Gröbner basis for <f1,f1>: [x^3 - 2*x*y, x^2*y - 2*y^2 + x, x^2, x*y, y^2 - 1/2*x]
This is a Gröbner basis: True


In [28]:
# extended testing (no input needed)
orders = ['lex','deglex','degrevlex']
tests = 10
polys_per_test = 5
for i in range(tests):
    poly_list = []
    for j in range(polys_per_test):
        poly_list.append(R.random_element())
    order = orders[random.randint(0,2)]
    G = groebner_basis(R,poly_list,order)
    if not is_groebner_basis(R,G,order):
        print('not a Gröbner basis')
        break
    if i==tests-1:
        print('all tests passed')

all tests passed


## Minimial Grobner Basis

**Theorem.** Let $I \not = \{0\}$ be a polynomial ideal. Then, for a given monomial ordering, $I$ has a reduced Gröbner basis, and the reduced Gröbner basis is unique.

**Definition** A *reduced Gröbner basis* for a polynomial ideal $I$ is a Gröbner basis $G$ for $I$ such that:
1. $LC(p)=1$ for all $p \in G$.
2. For all $p \in G$, no monomial of $p$ lies in $\langle LT(G  \setminus \{p\}) \rangle$.

In [29]:
R.<x,y> = PolynomialRing(QQ, order = 'deglex')
f1 = x^3 - 2*x*y
f2 = x^2*y - 2*y^2 + x
F = groebner_basis(R,[f1,f2], 'deglex')
print(f"Proposed Gröbner basis for <f1,f2>: {F}")
#res = is_reduced_groebner_basis(R,F,'grlex')
#print(f'Is minimal Gröbner basis found for <f1,f2>?: { res }')
G = minimal_groebner_basis(R,[f1,f2], 'deglex')
print(f"Proposed minimal Gröbner basis for <f1,f2>: {G}")
#res = is_reduced_groebner_basis(R,G,'grlex')
#print(f'Is minimal Gröbner basis found for <f1,f2>?: { res }')

Proposed Gröbner basis for <f1,f2>: [x^3 - 2*x*y, x^2*y - 2*y^2 + x, x^2, x*y, y^2 - 1/2*x]
Proposed minimal Gröbner basis for <f1,f2>: [x^2, x*y, y^2 - 1/2*x]


In [30]:
R.<x,y,z> = PolynomialRing(QQ, order = 'degrevlex')
f1 = x*y + z - x*z
f2 = x^2 - z
f3 = 2*x^3 - x^2*y*z - 1

F = groebner_basis(R,[f1,f2,f3], 'degrevlex')
print(f"Proposed Gröbner basis for <f1,f2,f3>: {F}")
G = minimal_groebner_basis(R,[f1,f2,f3], 'grlex')
print(f"Proposed minimal Gröbner basis for <f1,f2,f3>: {G}")
G = [x^6 - x^5 - 2*x^3 + 1, y - x^2 + x, z - x^2]

Proposed Gröbner basis for <f1,f2,f3>: [x^2*y*z - 2*x^3 + 1, z^4 - 3*z^3 - 4*y*z + 2*z^2 + x + z - 2, y^2*z - 2*y*z^2 + z^3 - z^2, y*z^2 + 2*y*z - 2*z^2 + 1, x^2 - z, x*y - x*z + z, y^2 - 2*y*z + z^2 - z, x*z + y*z - z^2, x + y - z]
Proposed minimal Gröbner basis for <f1,f2,f3>: [z^4 - 3*z^3 - 4*y*z + 2*z^2 - y + 2*z - 2, y*z^2 + 2*y*z - 2*z^2 + 1, y^2 - 2*y*z + z^2 - z, x + y - z]


# Applications of Grobner Bases

## Ideal Membership

**Theorem** Let $G = \{g_1, \ldots , g_t\}$ be a Gröbner basis for an ideal $I \subset k[x_1, \ldots , x_n]$ and let $f \in k[x_1,\ldots,x_n]$. Then $f \in I$ if and only if the remainder on division of $f$ by $G$ is zero.


In [31]:
# Example 1
R.<x,y,z> = PolynomialRing(QQ, order = 'deglex')
f = -4*x^2*y^2*z^2 + y^6 + 3*z^5
f1 = x*z - y^2 
f2 = x^3 - z^2
print(f'Is f in the ideal?: { f in ideal(f1,f2) }')
res = in_ideal_gen_by(R, f, [f1, f2], 'deglex')
print(f'Is remainder == 0: { res }')

Is f in the ideal?: True
Is remainder == 0: True


In [32]:
# Example 2
f = x*y - 5*z^2 + z
print(f'Is f in the ideal?: { f in ideal(f1,f2) }')
res = in_ideal_gen_by(R, f, [f1, f2], 'deglex')

print(f'Is remainder == 0: { res }')

Is f in the ideal?: False
Is remainder == 0: False


In [33]:
# Example 2
R.<x,y,z> = PolynomialRing(QQ, order = 'deglex')
f = x^2 - 2*x
poly_list = [x^2+y, z-x, z-y/2]
print(f'Is f in the ideal?: { f in ideal(poly_list) }')
res = in_ideal_gen_by(R, f, poly_list, 'deglex')
print(f'Is remainder == 0: { res }')

Is f in the ideal?: False
Is remainder == 0: False


In [34]:
# Example 3
R.<x,y,z> = PolynomialRing(QQ, order = 'deglex')
f = x^2 - 2*x
poly_list = [x^2+y, z+x, z-y/2]
print(f'Is f in the ideal?: { f in ideal(poly_list) }')
res = in_ideal_gen_by(R, f, poly_list, 'deglex')
print(f'Is remainder == 0: { res }')

Is f in the ideal?: True
Is remainder == 0: True


## Elimination Theorem

**Theorem (The Elimination Theorem).** Let $I \subset k[x_1,\ldots,x_n]$ be an ideal and let $G$ be a Gröbner basis of $I$ with respect to **lex order** where $x_1 > x_2 > \ldots > x_n$. Then, for every $0 \leq l \leq n$, the set
$$G_l = G \cap k[x_{l + 1}, \ldots, x_n]$$
is a Gröbner basis of the $l$-th elimination ideal $I_l$.

Use elimination to solve the system
\begin{align*}
0 &= f_1 = x^2+ 2y^2 - y- 2z \\
0 &= f_2 = x^2 - 8y^2 + 10z - 1  \\
0 &= f_3 = x^2 - 7yz
\end{align*}
How many solutions are there in $\mathbb R^3$; how many are there in $\mathbb C^3$?

In [35]:
R.<x,y,z> = PolynomialRing(QQ, order = 'lex')
f1 = x^2+ 2*y^2 - y- 2*z 
f2 = x^2 - 8*y^2 + 10*z - 1 
f3 = x^2 - 7*y*z

G = minimal_groebner_basis(R,[f1,f2,f3], 'lex')
print(G)
print(len(G))

[x^2 - 392/9*z^2 + 28/3*z - 7/9, y - 490/9*z^2 + 67/6*z - 13/18, z^3 - 313/980*z^2 + 37/980*z - 1/490]
3


Use elimination to solve the system
\begin{align*}
0 &= f_1 = x^2+ y + z - 1 \\
0 &= f_2 = x + y^2 + z - 1 \\
0 &= f_3 = x + y + z^2 - 1
\end{align*}
How many solutions are there in $\mathbb R^3$; how many are there in $\mathbb C^3$?

In [36]:
R.<x,y,z> = PolynomialRing(QQ, order = 'lex')
f1 = x^2 + y + z - 1
f2 = x + y^2 + z - 1
f3 = x + y + z^2 - 1

G = minimal_groebner_basis(R,[f1,f2,f3], 'lex')
print(G)

[x + y + z^2 - 1, y^2 - y - z^2 + z, y*z^2 + 1/2*z^4 - 1/2*z^2, z^6 - 4*z^4 + 4*z^3 - z^2]


Use elimination to solve the system
\begin{align*}
0 &= f_1 = x^5 + y^2 + z^2 - 4 \\
0 &= f_2 = x^2 + 2y^2 - 5 \\
0 &= f_3 = xz - 1
\end{align*}
How many solutions are there in $\mathbb R^3$; how many are there in $\mathbb C^3$?

In [38]:
R.<x,y,z> = PolynomialRing(QQ, order = 'lex')
f1 = x^5+y^2+z^2-4
f2 = x^2+2*y^2-5
f3 = x*z-1

G = minimal_groebner_basis(R,[f1,f2,f3], 'lex')
print(G)

[x + z^6 - 3/2*z^4 - 1/2*z^2, y^2 - 1/2*z^5 + 3/4*z^3 + 1/4*z - 5/2, z^7 - 3/2*z^5 - 1/2*z^3 + 1]


Use elimination to solve the system
\begin{align*}
0 &= f_1 = x^2+ - 2xz + 5 \\
0 &= f_2 = xy^2 + yz + 1 \\
0 &= f_3 = 3y^2 - 8xz
\end{align*}
How many solutions are there in $\mathbb R^3$; how many are there in $\mathbb C^3$?

In [39]:
R.<x,y,z> = PolynomialRing(QQ, order = 'lex')
f1 = x^2 - 2*x*z + 5
f2 = x*y^2 + y*z + 1
f3 = 3*y^2 - 8*x*z

G = minimal_groebner_basis(R,[f1,f2,f3], 'lex')
print(G)

[x + 119960726732800/483596973*z^7 + 2927549440000/161198991*z^6 - 157608191920640/161198991*z^5 - 611927395721600/483596973*z^4 - 14857079919551480/1450790919*z^3 + 377848832224480/161198991*z^2 - 3177979995598/17910999*z + 240873804800/53732997, y - 46473973780480/483596973*z^7 - 1147408998400/161198991*z^6 + 61057669722944/161198991*z^5 + 237223375813760/483596973*z^4 + 5756399991700688/1450790919*z^3 - 145833810707104/161198991*z^2 + 135421782400/1990111*z - 91569997880/53732997, z^8 - 1263/320*z^6 - 77/16*z^5 - 157093/3840*z^4 + 3991/320*z^3 - 45/32*z^2 + 9/128*z - 27/20480]


Use elimination to solve the system
\begin{align*}
0 &= f_1 = x^2+ y^2 + z^2 - 2x \\
0 &= f_2 = x^3 - yz - x \\
0 &= f_3 = x - y + 2z
\end{align*}
How many solutions are there in $\mathbb R^3$; how many are there in $\mathbb C^3$?

In [ ]:
R.<x,y,z> = PolynomialRing(QQ, order = 'lex')
f1 = x^2+ y^2 + z^2 - 2*x
f2 = x^3 - y*z - x 
f3 = x - y + 2*z
G = minimal_groebner_basis(R,[f1,f2,f3], 'lex')
print(G)

In [42]:
R.<x,y> = PolynomialRing(QQ, order = 'lex')
f1 = 0.5*x^2 + 3*y^2 - 1
f2 = 3*x^2 + 0.5*y^2 - 1
G = minimal_groebner_basis(R,[f1,f2], 'lex')
print(G)

[x^2 - 0.285714285714286, y^2 - 0.285714285714286]


# Want to learn more? 

https://www.siam.org/conferences/cm/conference/ag23